In [ ]:
from flask import Flask, jsonify, request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def hello_world():
    return "Hello World! <strong>I am learning Flask</strong>", 200

In [ ]:
@app.route("/", methods = ['POST'])
def apicall():
     """API Call

    Pandas dataframe (sent as a payload) from API Call
    """
    try:
        test_json = request.get_json()
        test = pd.read_json(test_json, orient='records')

        #To resolve the issue of TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'
        test['Dependents'] = [str(x) for x in list(test['Dependents'])]

        #Getting the Loan_IDs separated out
        loan_ids = test['Loan_ID']

    except Exception as e:
        raise e

    clf = 'model_v1.pkl'
    vocab = 'model_vocabulary.pkl'
    if test.empty:
        return(bad_request())
    else:
        #Load the saved model
        print("Loading the model...")
        loaded_model = None
        with open(clf,'rb') as f:
            loaded_model = pickle.load(f)
            
        print("Loading the vocabulary...")
        loaded_vocabulary = None
        with open(vocab,'rb') as f2:
            loaded_vocabulary = pickle.load(f2)
        
        vectorizer= CountVectorizer(vocabulary=loaded_vocabulary)
        teste_predict_vect = vectorizer.transform(test) 
        
        print("The model and vocabulary has been loaded...doing predictions now...")
        predictions = loaded_model.predict(teste_predict_vect)

        """Add the predictions as Series to a new pandas dataframe
                                OR
           Depending on the use-case, the entire test data appended with the new files
        """
        prediction_series = list(pd.Series(predictions))

        final_predictions = pd.DataFrame(list(zip(loan_ids, prediction_series)))

        """We can be as creative in sending the responses.
           But we need to send the response codes as well.
        """
        responses = jsonify(predictions=final_predictions.to_json(orient="records"))
        responses.status_code = 200

        return (responses)

In [ ]:
filename = 'model_v1.pkl'

In [ ]:
with open(filename ,'rb') as f:
    loaded_model = joblib.load(f)

In [ ]:
vocab_filename = 'model_vocabulary.pkl'

In [ ]:
with open(vocab_filename, 'rb') as f2:
    loaded_vocabulary = joblib.load(f2)

In [ ]:
print(loaded_model.classes_)

In [ ]:
vectorizer_train = CountVectorizer(vocabulary=loaded_vocabulary)

In [ ]:
teste_predict=['EPSON MAGENTO','LILLO BICO','BOV AMERICANA KG','KING 100ML JASMIM']
teste_predict_vect = vectorizer_train.transform(teste_predict) 
loaded_model.predict(teste_predict_vect)

In [ ]:
app.run()